In [ ]:
# -*- coding: utf-8 -*-
'''
论文题目：基于深度学习的PSA过程优化与控制
作    者：余秀鑫
单    位：天津大学化工学院
时    间：2022.04.12
'''

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import os
import time
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

In [4]:
def mse(y_true, y_pred):
  return tf.metrics.mean_squared_error(y_true=tf.squeeze(y_true),
                                       y_pred=tf.squeeze(y_pred))

In [5]:
def R2_Adj(y_true, y_pred):
    n, k = 863, 7
    R2 = r2_score(y_true, y_pred)
    R2_Adj = 1 - (1 - R2) * (n - 1) / (n - k - 1)
    return R2_Adj

In [6]:
def create_model_checkpoint(model_name, save_path="model_saved"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                            verbose=0, # only output a limited amount of text
                                            save_best_only=True) # save only the best model to file

In [7]:
# keys = [
#     "Tem_AD", "Tem_CD", "Tem_BD", "Tem_VU", "Tem_FP", "q_CO2_AD", "q_CO2_CD",
#     "q_CO2_BD", "q_CO2_VU", "q_CO2_FP", "q_H2_AD", "q_H2_CD", "q_H2_BD",
#     "q_H2_VU", "q_H2_FP", "y_CO2_AD", "y_CO2_CD", "y_CO2_BD", "y_CO2_VU",
#     "y_CO2_FP", "y_H2_AD", "y_H2_CD", "y_H2_BD", "y_H2_VU", "y_H2_FP"
# ]
keys = ['Tem_VU']

In [8]:
for key in keys:
    thisdata = key
    print(thisdata + ' Started')
    x_train = np.load("../../results_of_sample1000\DL\Input.npy")
    y_train = np.load("../../results_of_sample1000\DL/" + thisdata + ".npy")

    scaler_x = MinMaxScaler(feature_range=(0, 1)).fit(x_train)
    x_train_std = scaler_x.transform(x_train)
    scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(y_train)
    y_train_std = scaler_y.transform(y_train)

    np.random.seed(666)  # 使用相同的seed，保证输入特征和标签一一对应
    np.random.shuffle(x_train)
    np.random.seed(666)
    np.random.shuffle(x_train_std)
    np.random.seed(666)
    np.random.shuffle(y_train)
    np.random.seed(666)
    np.random.shuffle(y_train_std)
    tf.random.set_seed(666)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(49, activation="sigmoid"),
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                           beta_1=0.9,
                                           beta_2=0.999),
        loss=tf.keras.losses.mse,
    )

    history = model.fit(
        x_train_std,
        y_train_std,
        batch_size=64,
        epochs=2000,
        validation_split=0.2,
        validation_freq=1,
        verbose=0,
    )
    
    model.save('model_saved/'+ thisdata)
    
    print(thisdata + ' Saved')

Tem_VU Started
INFO:tensorflow:Assets written to: model_saved/Tem_VU\assets
Tem_VU Saved
